# 1. Data Validation

In [7]:
# Basic imports

import json
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from dotenv import load_dotenv
load_dotenv('.env')
import os
import plotly.graph_objects as go

from dotenv import load_dotenv
import os


In [8]:
# Build SparkSession
spark = SparkSession.builder.appName("DataPreparation").getOrCreate()
base_path = os.getenv('BASE_PATH')

# 2. DATA INGESTION

In [9]:
# Data to read - offers.csv
data_dir_offers = f'{base_path}/offers.csv.gz'
data_file_offers = data_dir_offers

! head $data_file_offers

# Data to read - sampleSubmission.csv
data_dir_sampleSubmission = f'{base_path}/sampleSubmission.csv.gz'
data_file_sampleSubmission = data_dir_sampleSubmission

! head $data_file_sampleSubmission

# Data to read - testHistory.csv
data_dir_testHistory = f'{base_path}/testHistory.csv.gz'
data_file_testHistory = data_dir_testHistory

! head $data_file_testHistory

# Data to read - trainHistory.csv
data_dir_trainHistory = f'{base_path}/trainHistory.csv.gz'
data_file_trainHistory = data_dir_trainHistory

! head $data_file_trainHistory

# Data to read - transactions.csv
data_dir_transactions = f'{base_path}/transactions.csv.gz'
data_file_transactions = data_dir_transactions

! head $data_file_transactions;

�}ESoffers.csv ���n�0��=Q�����u�0`K� ���������Y)�^__��|��|?�����p���\������x���=�|{�������fW�q.g�brjm��Q7� )��u�X�t)XG&fa�O�$j9�(���׶#A��䜘�Bw��
,d�2�3;�b4[&� ̞E��f�l<�Ȉ���D�"��#u���V҃��Y�@���ʎ��W;XKU�C�6� ���5j�T���g��̛�ܮA�U=@AV0',%�W��M�=Sp!��^�����9�{�h�Ȭ��o�F#F�|l�W�Yխ�X���#
�H�l*j#����A�ø�8f���Br��Eh�%C�-��r���	��+���+C���	RL�5B���r�V  head: cannot open 'data/sampleSubmission.csv.gz' for reading: No such file or directory
�.��testHistory.csv |�[�$5��YK!�r�c/qb�"���G�4��N��������?����?����o����������o�,MWǥ����e�����!H��C�fԃ�C��������	�C�!;/��"xk�oԅH�K�Z�n@�E�.� D� #��k��V}�QT�4^@߀
�+`��:� "׽E����B,��V��
| 4�����	PQ%��������^�ߧ��]Z�s��Kch�/Ľě��E_�p1���VG���Y9^�ᅐ��z| ��}�%\�NJ� ���(}�1T�=��t��_\�p�K�J��8 �|~��x�K
����i�@�T���مC���5PR�%	� 4���^9DzF|�n�ЦQk���ȯԤ]���]L����+���#Q^��TT�E�e����1����(���n�G:�Rp�u�8�IE�X~�0�����5��,���'䪷��#Ǚa����ն��Gζ*Y���|�ɾm+���C{���R���A!c- EĊT�4��

**Offers**

In [10]:
# Reading data - offers.csv
df_offers = spark.read.csv(
        data_file_offers, 
        header=True, sep=',', inferSchema=True
    )

**TESTHISTORY**

In [12]:
# Reading data - testHistory.csv
df_testHistory = spark.read.csv(
        data_file_testHistory, 
        header=True, sep=',', inferSchema=True
    )

**TRAINHISTORY**

In [13]:
# Reading data - trainHistory.csv
df_trainHistory = spark.read.csv(
        data_file_trainHistory, 
        header=True, sep=',', inferSchema=True
    )

**TRANSACTIONS**

In [14]:
df_transactions = spark.read.csv(
    data_file_transactions, 
    header=True, sep=',', inferSchema=False
).sample(fraction=0.001, seed=42).limit(1000000)

**KAFKA-CLUSTER-CREATE-TOPIC**

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic

my_topic = 'shoppers'
try:
    admin = KafkaAdminClient(bootstrap_servers='localhost:9092')
    topic = NewTopic(name=my_topic, num_partitions=1, replication_factor=1)
    admin.create_topics([topic])
    print(f'DEBUG: Topic {my_topic} successfully created.')
except Exception as e:
    print(f'Error creating topic.\n{e}')

**KAFKA-CLUSTER-INFO**

In [ ]:
from kafka import KafkaAdminClient
from kafka.cluster import ClusterMetadata

try:
    admin = KafkaAdminClient(bootstrap_servers='localhost:9092')
    print(f'List of topics in the cluster: {admin.list_topics()}')
    print(f'List of consumer groups known to the cluster: {admin.list_consumer_groups()}')
except Exception:
    print(f'Error connecting to cluster')

clusterMetadata = ClusterMetadata(bootstrap_servers=['localhost:9092'])
print(f'All brokers metadata: {clusterMetadata.brokers()}')
print(f'Partitions for topic shoppers: {clusterMetadata.partitions_for_topic("shoppers")}')
print(f'Topics: {clusterMetadata.topics()}')

**KAFKA-PRODUCER**

In [ ]:
import os, time, json, datetime, csv
import pandas as pd
from kafka import KafkaProducer

topic = 'shoppers'
inputfile = './data/validated_testHistory.csv'  # Use test data for streaming
chunksize = 10000
sleeptime = 0.6

def datetime_converter(dt):
    if isinstance(dt, datetime.datetime):
        return dt.__str__()

def on_send_success(metadata):
    print(f'Published to Topic: {metadata.topic}, Partition: {metadata.partition}, Offset: {metadata.offset}')

def on_send_error(excp):
    print(f'Error: {excp}')

kafka_producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    api_version=(3, 9),
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

# Load and join test data
test_df = pd.read_csv(inputfile)
offers_df = pd.read_csv('./data/validated_offers.csv')
transaction_agg = pd.read_parquet('./data/transaction_features.parquet')
df_joined = test_df.merge(offers_df, on='offer', how='left').merge(transaction_agg, on='id', how='left')

for _, row in df_joined.iterrows():
    json_data = json.dumps(row.to_dict(), default=datetime_converter)
    print(f'Sending to Kafka: {json_data}')
    try:
        kafka_producer.send(topic, value=json_data).add_callback(on_send_success).add_errback(on_send_error)
    except Exception as e:
        print(f'Error: {e}')
    time.sleep(sleeptime)

kafka_producer.flush()
kafka_producer.close()

**KAFKA-CONSUMER**

In [ ]:
from kafka import KafkaConsumer

topic = 'shoppers'
kafka_consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    api_version=(3, 9),
    auto_offset_reset='earliest',
    enable_auto_commit=False
)

kafka_consumer.subscribe([topic])
print(f"Listening on topic: {topic}")

for msg in kafka_consumer:
    print(f"Received: [{msg.topic}:{msg.partition}:{msg.offset}] value={msg.value.decode('utf-8')}")

**APP_SPARK**

In [ ]:
import time

# Importing critical functions that deal with data stream
from data_streaming import ( spark_initialize, data_stream_spark, 
            show_status, show_tables, show_sink_table, get_table_dataframe )

brokers = 'localhost:9092'
topic = 'books-amazon'
table = 'bookstable'

# Showing results of data stream processing
def show_spark_results(spark, table):
    df = get_table_dataframe(spark, table)
    # cols_interest = ['timestamp','Asin','Group','Format','Title','Author','Publisher']
    
    sleeptime = 0.8
    maxiterations = 30
    top_authors = 20
    top_publishers = 20

    # Iterative update
    for i in range(maxiterations):
        time.sleep(sleeptime)
        print(f'Processing...  Iteration {i} with in-between delay of {sleeptime} second(s)')
        print(f'Number of records processed so far: {df.count()}.')

        #df.select(cols_interest).show(truncate=False)
        #df.show(5, truncate=False)
        #show_sink_table(spark, table)

        print('Aggregated information as it stands (top 20):')
        df.groupBy('Author').count().orderBy('count', ascending=False).limit(top_authors).show(truncate=False)
        df.groupBy('Publisher').count().orderBy('count', ascending=False).limit(top_publishers).show(truncate=False)
        #df.groupBy('Group', 'Format').count().show(truncate=False)
        

# Execution

spark = spark_initialize()
query = data_stream_spark(spark, brokers, topic, table)

show_status(spark, query)
show_tables(spark)
show_sink_table(spark, table)
show_spark_results(spark, table)


**APP_STREAMLIT**

In [ ]:
import streamlit as st
import time

# Importing critical functions that deal with data stream (Spark/Kafka side)
from data_streaming import ( spark_initialize, data_stream_spark, 
                show_tables, show_status, get_table_dataframe )

# Caching the function that will access the running 
# Spark/Kafka data query (a DataFrame)
@st.cache_resource
def get_data():
    return get_table_dataframe(st.session_state.spark, st.session_state.table)

# Showing results of data stream processing, 
# as long as there is a SparkSession running
def results():

    if 'spark' not in st.session_state:
        return
    
    status_text = st.empty()
    progress_bar = st.progress(0)
    placeholder = st.empty()
    sleeptime = 0.8
    maxiterations = 30
    top_authors = 20
    top_publishers = 20

    # Iterative update
    for i in range(maxiterations):
        time.sleep(sleeptime)
        # getting data at this point in time
        df = get_data()
        count = df.count()
        status_text.warning(f'Processing...  Iteration {i} with in-between delay of {sleeptime} second(s). Messages/records processed so far: {count}.')
        cols1 = ['Author'] 
        df_author = df.groupBy(cols1).count().orderBy('count', ascending=False).limit(top_authors).toPandas()
        cols2 = ['Publisher']
        df_publisher = df.groupBy(cols2).count().orderBy('count', ascending=False).limit(top_publishers).toPandas()
        print(df_publisher)

        with placeholder.container():

            # Each chart in one column, so two columns required
            fig_col1, fig_col2 = st.columns(2)
            with fig_col1:
                st.markdown('### Author')
                st.markdown(f'**Counting of books by author - Top {top_authors}**')
                st.bar_chart(data=df_author, y='count', x=cols1[0], horizontal=True)
            with fig_col2:
                st.markdown('### Publisher')
                st.markdown(f'**Counting of books by publisher - Top {top_publishers}**')
                st.bar_chart(data=df_publisher, y='count', x=cols2[0], horizontal=True)

            # Show the related dataframes
            st.markdown('### Detailed tables view')
            st.markdown('**Author**')
            st.dataframe(df_author)
            st.markdown('**Publisher**')
            st.dataframe(df_publisher)
    
        progress_bar.progress(i)
  
    progress_bar.empty()
    status_text.success(f'Final results are shown after processing {count} messages/records.')

# Page to hold results
def page_results():
    st.empty()
    st.header(':one: Data stream processing')
    st.subheader('Results')
    results()
    
# Page to hold information about the app
def page_about():
    st.empty()
    st.header(':two: About')
    st.subheader('Lab class handout #6')
    st.write('Data streaming with Apache Spark and Apache Kafka')
    st.badge('Streamlit version', icon='ℹ️', color='blue')
    
# Entry point
def main():
    
    # Page config
    st.set_page_config(
        page_title = 'Books data streaming',
        initial_sidebar_state = 'expanded',
        layout = 'wide'
    )
    # App title
    st.title('Books data streaming')
    st.divider()
    with st.sidebar:
        st.empty()
        st.header('Algoritmos para Big Data')

    brokers = 'localhost:9092'
    topic = 'books-amazon'
    table = 'bookstable'

    # As code is running everytime the user interacts with, 
    # we must make sure that the spark side only starts once

    if 'spark' not in st.session_state:
        spark = spark_initialize()
        query = data_stream_spark(spark, brokers, topic, table)
        st.session_state.spark = spark
        st.session_state.table = table
        # just to check in the terminal
        show_status(spark, query)
        show_tables(spark)
        
    pages = [ st.Page(page_results, title='Results'),
              st.Page(page_about, title='About'),
            ]
    pg = st.navigation(pages)
    pg.run()

# Execution
if __name__ == "__main__":
    main()


**DATA_STREAMING**

In [ ]:
import os, sys, time, json
import pyspark
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T

def spark_initialize() -> SparkSession:
    scala_version = '2.12'
    spark_version = '3.3.1'
    packages = [
        f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
        f'org.apache.spark:spark-token-provider-kafka-0-10_{scala_version}:{spark_version}',
        f'org.apache.spark:spark-streaming-kafka-0-10_{scala_version}:{spark_version}',
        'org.apache.kafka:kafka-clients:3.3.1',
        'org.apache.commons:commons-pool2:2.8.0'
    ]
    spark = SparkSession.builder\
        .appName('Streaming')\
        .config('spark.jars.packages', ','.join(packages))\
        .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

def data_stream_spark(spark, brokers, topic, table) -> DataFrame:
    df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", brokers) \
        .option("subscribe", topic) \
        .option("includeHeaders", "true") \
        .option("startingOffsets", "earliest") \
        .load()

    spark.sql(f'drop table if exists {table}')
    query = df.writeStream \
        .queryName(f'{table}') \
        .outputMode("append") \
        .format("memory") \
        .start()
    return query

def show_status(spark, query):
    print(f'Active: {spark.streams.active[0].isActive}.')
    print(f'Status: {query.status}.')

def show_tables(spark):
    spark.sql("show tables").show(truncate=False)

def show_sink_table(spark, table):
    spark.sql(f'select * from {table}').show(truncate=False)

def get_table_dataframe(spark, table):
    df_kafka = spark.sql(f'select CAST(value AS STRING), topic, timestamp from {table}')
    schema = T.StructType([
        T.StructField("id", T.StringType(), True),
        T.StructField("offer", T.StringType(), True),
        T.StructField("category", T.StringType(), True),
        T.StructField("quantity", T.DoubleType(), True),
        T.StructField("avg_purchase", T.DoubleType(), True),
        T.StructField("transaction_count", T.DoubleType(), True),
        T.StructField("unique_categories", T.DoubleType(), True)
    ])
    df_kafka = df_kafka.withColumn('jsonvalue', F.from_json(F.col('value'), schema)) \
        .select(
            F.col('jsonvalue.id').alias('id'),
            F.col('jsonvalue.offer').alias('offer'),
            F.col('jsonvalue.category').alias('category'),
            F.col('jsonvalue.quantity').alias('quantity'),
            F.col('jsonvalue.avg_purchase').alias('avg_purchase'),
            F.col('jsonvalue.transaction_count').alias('transaction_count'),
            F.col('jsonvalue.unique_categories').alias('unique_categories'),
            F.col('timestamp')
        )
    return df_kafka